# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore') 

# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,heighington,53.2124,-0.4590,4.65,89,100,3.09,GB,1700999133
1,carnarvon,-24.8667,113.6333,22.20,71,85,7.31,AU,1700999227
2,ataq,14.5377,46.8319,29.22,19,5,4.30,YE,1700999229
3,iqaluit,63.7506,-68.5145,-10.15,73,75,1.54,CA,1700999231
4,port-aux-francais,-49.3500,70.2167,3.85,68,73,10.39,TF,1700999233


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [40]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    size="Humidity"
)

# Display the map
%capture c --no-display 
map_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [41]:
# Narrow down cities that fit criteria and drop any results with null values
idealweather_df= city_data_df.loc[(city_data_df["Cloudiness"]<=0)
                                  &(city_data_df["Max Temp"]<27)
                                  &(city_data_df["Max Temp"]>21)
                                  &(city_data_df["Wind Speed"]<5)
                                  &(city_data_df["Humidity"]<80)]    



# Drop any rows with null values
idealweather_df.dropna()

# Display sample data
idealweather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,ghat,24.9647,10.1728,22.28,23,0,2.05,LY,1700999372
140,bubaque,11.2833,-15.8333,26.44,64,0,2.07,GW,1700999482
202,vila velha,-20.3297,-40.2925,23.97,75,0,3.58,BR,1700999598
547,veinticinco de mayo,-35.4323,-60.1727,25.55,60,0,3.58,AR,1701000266


### Step 3: Create a new DataFrame called `hotel_df`.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
 
hotel_df=idealweather_df[["City","Country","Lat","Lng","Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

#hotel_df["Hotel Name"]="-----"
hotel_df.insert(5, "Hotel Name", "------") 

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
81,ghat,LY,24.9647,10.1728,23,------
140,bubaque,GW,11.2833,-15.8333,64,------
202,vila velha,BR,-20.3297,-40.2925,75,------
547,veinticinco de mayo,AR,-35.4323,-60.1727,60,------


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
params = {"apiKey":geoapify_key,
          "categories":categories}
         

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Latitude=row["Lat"]
    Longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL (provided)
    base_url = "https://api.geoapify.com/v2/places"
    


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghat - nearest hotel: فندق تاسيلي
bubaque - nearest hotel: Chez Julio
vila velha - nearest hotel: Hotel Prainha
veinticinco de mayo - nearest hotel: Jorly


,City,Country,Lat,Lng,Humidity,Hotel Name
81,ghat,LY,24.9647,10.1728,23,فندق تاسيلي
140,bubaque,GW,11.2833,-15.8333,64,Chez Julio
202,vila velha,BR,-20.3297,-40.2925,75,Hotel Prainha
547,veinticinco de mayo,AR,-35.4323,-60.1727,60,Jorly


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    size="Humidity",
    hover_cols = ["City", "Country","Lat","Lng","Humidity","Hotel Name"]
    )

# Display the map
%capture c --no-display 
hotel_map